In [21]:
import keras 
from keras.layers import Dense,Dropout, Conv2D,MaxPooling2D , Activation, Flatten
from keras.models import Sequential
from tensorflow.keras.layers import *
import numpy as np 
# import pandas as pd
import tensorflow as tf

In [22]:
model = Sequential()

model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())

model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))

model.add(Dense(units=3, activation="softmax"))
model.load_weights("../input/multi-class-freshness-classifier/rottenvsfresh_multi.h5")


In [23]:
import socket
from _thread import *
import threading
import json
from io import StringIO
from keras.preprocessing import image
import base64

class ModelWrapper:
    @staticmethod
    def detectFreshness(c):
        info = c.recv(4096)
        data = ''
        data.append(info)
        while (len(info) > 0):
#         print(results)
            info = c.recv(4096)
            data.append(info)
        data = json.loads( data.decode("utf-8") )
        print(data)
        img_width, img_height = 224, 224
#         img = image.load_img('image_path/image_name.jpg', target_size = (img_width, img_height))
        img = image.load_img(BytesIO(base64.b64decode(data['image'])), target_size=(224, 224))
#         img = data["image"].decode('base64'), target_size = (img_width, img_height))
        img = image.img_to_array(img)/255.
        img = np.expand_dims(img, axis = 0)
        
        prediction = model.predict(img)
        print(prediction)
        label = np.argmax(prediction, axis = 1)
        if label == 0:
            print( "Data" , json.dumps({"label": label }) )
            c.send( json.dumps({"label": "fresh" }).encode('utf-8') )
        else if label == 1:
            print( "Data" , json.dumps({"label": label }) )
            c.send( json.dumps({"label": "medium" }).encode('utf-8') )
        else if label == 2:
            print( "Data" , json.dumps({"label": label }) )
            c.send( json.dumps({"label": "rotten" }).encode('utf-8') )
        else:
            c.send( json.dumps({"label": 1 }).encode('utf-8') )
        c.close()

In [24]:

port = 5000
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.close()
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(( "0.0.0.0" , port ))
print("socket binded to port", port )
s.listen(5)
print("socket is listening")
all_conn = []
while True:
    conn, addr = s.accept()
    print('Connected to :', addr[0], ':', addr[1])
    all_conn.append( conn )
    start_new_thread(ModelWrapper.detectFreshness, (s,conn,))
    s.close()
    break
s.close()

OSError: [Errno 98] Address already in use